In [1]:
#Step 1: Install the Necessary Packages to Setup Your Environment
#We’ll need crewai, crewai_tools, langchain_openai and litellm
#!pip install crewai crewai_tools langchain_openai
#!pip install “litellm>=1.51.0”
#!pip install -U crewai

In [ ]:
import os
# Remove any leftover OpenAI / Google configuration in this process
for var in ["OPENAI_API_KEY", "OPENAI_BASE_URL", "GEMINI_API_KEY","GEMMA_MODEL_NAME"]:
    os.environ.pop(var, None)


In [1]:

#Step 2: Set Up Your Environment Variables
#We will use llama-3.3–70b-versatile served by GROQ. So you have to obtain your API key from GROQ

import os
#from langchain_openai import ChatOpenAI
from crewai import LLM

os.environ["GROQ_API_KEY"] = "Get Your API Key from GROQ"
print(os.getenv("GROQ_API_KEY"))

GROQ_API_KEY = os.environ["GROQ_API_KEY"]  #It will raise error if not set

# Use Groq’s OpenAI-compatible endpoint
GROQ_BASE_URL = os.getenv("GROQ_OPENAI_BASE", "https://api.groq.com/openai/v1")

# Choose a Llama 3 model served by Groq
llm = LLM(
    model="groq/llama-3.3-70b-versatile",  # Groq provider + Llama 3 model
    # You can also set extra params if you like:
    temperature=0.2,
    max_completion_tokens=2048,
)

ModuleNotFoundError: No module named 'crewai'

In [ ]:
#Step 3: Build a Custom BioRxiv Search Tool
#While CrewAI has built-in tools, we'll create a custom one to search BioRxiv for specific categories on a given date.
#We inherit from CrewAI's BaseTool class to define our tool.
from crewai.tools import BaseTool
from pydantic import BaseModel, Field
from typing import Type, List
import datetime, requests

#import biorxiv_api # A hypothetical library for the BioRxiv API

BIORXIV_SUBJECTS = {
    "immunology": "immunology",
    "microbiology": "microbiology",
    "genomics": "genomics",
    "neuroscience": "neuroscience",
    "cancer_biology": "cancer_biology",
    "bioinformatics": "bioinformatics",
    # add more from biorxiv's subject list as needed
}
class FetchBioRxivPapersInput(BaseModel):
    start_date: datetime.date = Field(
        ...,
        description="Start date (UTC) of the search window, e.g. 2025-11-01.",
    )
    end_date: datetime.date = Field(
        ...,
        description="End date (UTC) of the search window, inclusive, e.g. 2025-11-09.",
    )
    subject_key: str = Field(
        ...,
        description=f"One of: {list(BIORXIV_SUBJECTS.keys())}",
    )

class FetchBioRxivPapersTool(BaseTool):
    name: str = "fetch_biorxiv_papers"
    description: str = "Fetches all bioRxiv preprints for a given subject and  date."
    args_schema: Type[BaseModel] = FetchBioRxivPapersInput

    def _run(self, start_date: datetime.date, end_date: datetime.date,subject_key: str,) -> List[dict]:

        # Normalise subject
        subj = BIORXIV_SUBJECTS.get(subject_key.lower(), "")

        # Ensure start_date <= end_date
        if end_date < start_date:
            start_date, end_date = end_date, start_date

        start_str = start_date.strftime("%Y-%m-%d")
        end_str = end_date.strftime("%Y-%m-%d")

        # Set the API URL for 'biorxiv' server
        url = f"https://api.biorxiv.org/details/biorxiv/{start_str}/{end_str}/0"

        r = requests.get(url, timeout=30)
        r.raise_for_status()
        payload = r.json().get("collection", [])

        results: List[dict] = []
        for item in payload:

            # Add filtering for the subject ---
            item_category = item.get("category", "").lower()
            if item_category == subj:
                results.append(
                    {
                        "title": item.get("title"),
                        "authors": item.get("authors"),
                        "summary": item.get("abstract"),
                        "published": item.get("date"),
                        "url": f"https://www.biorxiv.org/content/{item.get('doi')}v{item.get('version')}",
                        "category": item.get("category"),
                    }
                )
        return results

# Instantiate the tool
biorxiv_search_tool = FetchBioRxivPapersTool()

In [ ]:
#1. Biomedical Scientist
from crewai import Agent

biomedical_scientist = Agent(
    role="Senior Biomedical Curator",
    goal=("From bioRxiv results between {start_date} and {end_date} in {subject}, "
      "identify the top 10 most relevant and methodologically solid preprints."),
    backstory=(
        "You are an experienced biomedical scientist who triages preprints, "
        "Your skill lies in quickly identifying groundbreaking research from preprints,"
        "separating true innovation from incremental updates."
    ),
    tools=[biorxiv_search_tool],
    llm=llm,
    verbose=True,
)

#2. The Frontend Engineer
frontend_engineer = Agent(
    role="Frontend Engineer",
    goal="Compile the research findings into a visually appealing and easy-to-read HTML file.",
    backstory=(
        "You are a skilled frontend engineer who specializes in creating clean, professional reports for scientific data. "
        "You excel at turning complex information into well-structured HTML that is both functional and aesthetic."
    ),
    llm=llm,
    verbose=True,
)

In [ ]:
#Step 5: Define the Tasks
#Our design has already broken the goal into smaller tasks. Each of the tasks is coded here and assigned to an agent.

#Task1 - Research Task
from crewai import Task

research_task = Task(
    description=(
        "Use `fetch_biorxiv_papers` to collect {subject} preprints from bioRxiv "
         "between {start_date} and {end_date},"
        "methods, data availability, appropriate controls, statistical rigor, and "
        "potential impact. Return the top 10 papers with title, authors, short abstract and link."
    ),
    expected_output=(
        "A ranked list of the top 10 papers, each containing:"
        "\n- Title"
        "\n- Authors"
        "\n- A concise summary of the abstract"
        "\n- A link to the paper on BioRxiv"
        "...for the period {{start_date}}–{{end_date}}, or a clear 'no results' message."
    ),
    agent=biomedical_scientist,
    human_input=True # Keep the interactive loop and allow feedback before finishing the task
)

#Task2 - Reporting Task
#This task takes the output of the research_task as its context.
reporting_task = Task(
    description="Compile the ranked list of papers into a detailed HTML report for {subject}, covering {start_date}–{end_date}.",
    expected_output="An HTML file saved to './biorxiv_report.html' containing a formatted report of the top 10 papers in the period {start_date}–{end_date} (or a 'no results' message).",
    agent=frontend_engineer,
    context=[research_task], # Use the output of the research_task
    output_file="./biorxiv_report.html", # Specify the output file
    human_input=True
)

In [ ]:
#Step 6: Assemble the Crew
#At this stage, the Agents and Tasks are brought together to form the crew.

from crewai import Crew, Process

biorxiv_research_crew = Crew(
    agents=[biomedical_scientist, frontend_engineer],
    tasks=[research_task, reporting_task],
    verbose=True,
    process=Process.sequential # Ensures tasks run one after the other
)

In [ ]:
#Step 7: Run the Crew!
# The crew is launched at this stage using the kickoff method,
# The date range to be searched is passed as input to the crew.
crew_inputs = {
    "start_date": "2025-11-25",  # Example start date
    "end_date": "2025-11-26",    # Example end date
    "subject": "microbiology"
}
result = biorxiv_research_crew.kickoff(inputs=crew_inputs)

print(result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: df84722c-7559-473f-b34c-4416c523ce5b                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Biomedical Curator                                                                               │
│                                                                                                                 │
│  Task: Use `fetch_biorxiv_papers` to collect microbiology preprints from bioRxiv between 2025-11-25 and         │
│  2025-11-26,methods, data availability, appropriate controls, statistical rigor, and potential impact. Return   │
│  the top 10 papers with title, authors, short abstract and link.                                                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Biomedical Curator                                                                               │
│                                                                                                                 │
│  Thought: Thought: To identify the top 10 most relevant and methodologically solid preprints in microbiology    │
│  from bioRxiv between 2025-11-25 and 2025-11-26, I need to first fetch all the preprints within this date       │
│  range and subject area. This will provide me with a list of potential candidates to evaluate based on their    │
│  methods, data availability, controls, statistical rigor, and potential impact.                                 │
│                                                                                                                 │
│  Using Tool: fetch_biorxiv_papers                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Biomedical Curator                                                                               │
│                                                                                                                 │
│  Thought: Thought: To identify the top 10 most relevant and methodologically solid preprints in microbiology    │
│  from bioRxiv between 2025-11-25 and 2025-11-26, I need to first fetch all the preprints within this time       │
│  frame and subject area. This will involve using the `fetch_biorxiv_papers` tool with the appropriate           │
│  parameters.                                                                                                    │
│                                                                                                                 │
│  Using Tool: fetch_biorxiv_papers                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

 ## Final Result: 1. Title: Tityus serrulatus venom: TsTX-κ beta a neurotoxin with anti-microbial activity and non-hemolytic
Authors: Oliveira, T. d. J.; Pereira, N.; Riciluca, K. C. T.; do Nascimento, S. M.; Castro de Oliveira, U.; Silva Junior, P. I.
Summary: The search for new molecules capable of inhibiting the growth of pathogens is a challenge for researchers, who find in venoms a rich source of biomolecules, including antimicrobial peptides (AMPs).
Link: https://www.biorxiv.org/content/10.1101/2022.10.09.511467v3

2. Title: RNA virus ecogenomics along a subarctic permafrost thaw gradient
Authors: Pratama, A. A.; Dominguez-Huerta, G.; Wainaina, J. M.; Bolduc, B.; Tian, F.; Ellenbogen, J.; Guo, J.; EMERGE Field Team,  ; EMERGE Coordinators,  ; Kuhn, J. H.; Wrighton, K. C.; Kusel, K.; Zayed, A. A.; Sullivan, M. B.
Summary: RNA virus diversity was structured by habitat (palsa, bog, and fen), and these patterns correlated with pH and carbon dioxide and methane emissions.
Link: https:



=====
## HUMAN FEEDBACK: Provide feedback on the Final Result and Agent's actions.
Please follow these guidelines:
 - If you are happy with the result, simply hit Enter without typing anything.
 - Otherwise, provide specific improvement requests.
 - You can provide multiple rounds of feedback until satisfied.
=====



╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Biomedical Curator                                                                               │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  1. Title: Tityus serrulatus venom: TsTX-κ beta a neurotoxin with anti-microbial activity and non-hemolytic     │
│  Authors: Oliveira, T. d. J.; Pereira, N.; Riciluca, K. C. T.; do Nascimento, S. M.; Castro de Oliveira, U.;    │
│  Silva Junior, P. I.                                                                                            │
│  Summary: The search for new molecules capable of inhibiting the growth of pathogens is a challenge for         │
│  researchers, who find in venoms a rich source of biomolecules, including antimicrobial peptides (AMPs).        │
│  Link: https://www.biorxiv.org/content/10.1101/2022.10.09.511467v3                                              │
│                                                                                                                 │
│  2. Title: RNA virus ecogenomics along a subarctic permafrost thaw gradient                                     │
│  Authors: Pratama, A. A.; Dominguez-Huerta, G.; Wainaina, J. M.; Bolduc, B.; Tian, F.; Ellenbogen, J.; Guo,     │
│  J.; EMERGE Field Team,  ; EMERGE Coordinators,  ; Kuhn, J. H.; Wrighton, K. C.; Kusel, K.; Zayed, A. A.;       │
│  Sullivan, M. B.                                                                                                │
│  Summary: RNA virus diversity was structured by habitat (palsa, bog, and fen), and these patterns correlated    │
│  with pH and carbon dioxide and methane emissions.                                                              │
│  Link: https://www.biorxiv.org/content/10.1101/2025.02.13.637936v4                                              │
│                                                                                                                 │
│  3. Title: Epimutations driven by RNAi or heterochromatin evoke transient antimicrobial drug resistance in      │
│  fungi                                                                                                          │
│  Authors: Son, Y.-E.; Perez-Arques, C.; Heitman, J.                                                             │
│  Summary: Antimicrobial resistance (AMR) is a global health threat emerging through microbe adaptation, driven  │
│  by genetic variation, genome plasticity or epigenetic process.                                                 │
│  Link: https://www.biorxiv.org/content/10.1101/2025.06.17.660219v4                                              │
│                                                                                                                 │
│  4. Title: Cell cycle dysregulation of globally important SAR11 bacteria resulting from environmental           │
│  perturbation                                                                                                   │
│  Authors: Cheng, C.; Bennett, B. D.; Savalia, P.; Asrari, H.; Biel, C.; Evans, K. A.; Tang, R.; Thrash, C.      │
│  Summary: Genome streamlining is hypothesized to occur in bacteria as an adaptation to resource-limited         │
│  environments, but can result in gene losses affecting fundamental aspects of cellular physiology.              │
│  Link: https://www.biorxiv.org/content/10.1101/2025.06.25.661606v3                                              │
│                                                                                                                 │
│  5. Title: The Phosphate-Specific Transport System Gene